In [1]:
!pip install jsonlines pytorch_pretrained_bert

    100% |████████████████████████████████| 122kB 11.4MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
PATH = '/content/gdrive/My Drive/Colab Notebooks/NLP/CommonsenseQA'

In [6]:
!wget https://www.dropbox.com/s/ocrxn41dqrj8d7a/bert.zip

--2019-03-17 19:28:32--  https://www.dropbox.com/s/ocrxn41dqrj8d7a/bert.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ocrxn41dqrj8d7a/bert.zip [following]
--2019-03-17 19:28:32--  https://www.dropbox.com/s/raw/ocrxn41dqrj8d7a/bert.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca4c07bca305d93bb3798e6ab7c.dl.dropboxusercontent.com/cd/0/inline/AdTzt-ksniMvIDfEpZT1e4GbV7lSAXJ-6bPwWl3zZJ9k5HNXkmBOjy1cUYTZ5kaQL1SKujobgvlIKJHg4CLBTED5VUhJFwJw2GeA0S1CHJykuDMEnZhGiUzNWkxf-qta-xI/file# [following]
--2019-03-17 19:28:32--  https://uca4c07bca305d93bb3798e6ab7c.dl.dropboxusercontent.com/cd/0/inline/AdTzt-ksniMvIDfEpZT1e4GbV7lSAXJ-6bPwWl3zZJ9k5HNXkmBOjy1cUYTZ5kaQL1SKujobgvlIKJHg4CLBTED5VUhJFwJw2GeA0S1CHJykuDMEnZhGiUzNWkx

In [7]:
!unzip bert.zip 

Archive:  bert.zip
   creating: bert/
  inflating: bert/bert_config.json   
  inflating: bert/eval_results.txt   
  inflating: bert/pytorch_model.bin  


In [9]:
# move model into folder
import shutil, os 
shutil.move('bert', PATH)

'/content/gdrive/My Drive/Colab Notebooks/NLP/CommonsenseQA/bert'

In [0]:
import sys
sys.path.append(PATH)

In [11]:
import os
from finetuning.bert import *
from tqdm import tqdm

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [0]:
data_dir = os.path.join(PATH, 'data')
output_dir = os.path.join(PATH, 'bert')
bert_model = 'bert-base-uncased'

CONFIG_NAME = 'bert_config.json'
WEIGHTS_NAME = 'pytorch_model.bin'

output_config_file = os.path.join(output_dir, CONFIG_NAME)
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)

In [0]:
parser = init_parser(PATH=PATH)
args = parser.parse_args('')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
config = BertConfig(output_config_file)
model = BertForMultipleChoice(config, num_choices=3)
model.load_state_dict(torch.load(output_model_file))
model.to(device);

In [16]:
tokenizer = BertTokenizer.from_pretrained(bert_model, do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 916655.43B/s]


In [0]:
eval_examples = read_qa_examples(os.path.join(data_dir, train_file), is_training = False)

In [0]:
Q = 'If a lantern is not for sale, where is it likely to be?'
A = 'antique shop'
B = 'house'
C = 'dark place'

In [0]:
example = CommonsenseQA(
                qn=Q.lower(),
                ans_0=A.lower(),
                ans_1=B.lower(),
                ans_2=C.lower(),
                label=0 # irrelvant
            )

In [0]:
features = convert_examples_to_features([example], tokenizer, 100, True)

In [0]:
all_input_ids = torch.tensor(select_field(features, 'input_ids'), dtype=torch.long)
all_input_mask = torch.tensor(select_field(features, 'input_mask'), dtype=torch.long)
all_segment_ids = torch.tensor(select_field(features, 'segment_ids'), dtype=torch.long)
all_label = torch.tensor([f.label for f in features], dtype=torch.long)
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label)

In [0]:
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=8)

model.eval();

In [38]:
ans_map = ['A','B','C']
for input_ids, input_mask, segment_ids, label_ids in eval_dataloader:
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
        logits = model(input_ids, segment_ids, input_mask)

    logits = logits.detach().cpu().numpy()
    print(ans_map[np.argmax(logits)])

B
